In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

books = pd.read_csv('books.csv', encoding = 'ISO-8859-1')
books.describe()

,id,book_id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,9.415000e+03,9979.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,5000.50000,5.264697e+06,5.471214e+06,8.646183e+06,75.712700,9.754393e+12,1981.987674,4.002191,5.400124e+04,5.968732e+04,2919.955300,1345.040600,3110.885000,11475.893800,1.996570e+04,2.378981e+04
std,2886.89568,7.575462e+06,7.827330e+06,1.175106e+07,170.470728,4.428246e+11,152.576665,0.254427,1.573700e+05,1.678038e+05,6124.378132,6635.626263,9717.123578,28546.449183,5.144736e+04,7.976889e+04
min,1.00000,1.000000e+00,1.000000e+00,8.700000e+01,1.000000,1.951703e+08,-1750.000000,2.470000,2.716000e+03,5.510000e+03,3.000000,11.000000,30.000000,323.000000,7.500000e+02,7.540000e+02
25%,2500.75000,4.627575e+04,4.791175e+04,1.008841e+06,23.000000,9.780000e+12,1990.000000,3.850000,1.356875e+04,1.543875e+04,694.000000,196.000000,656.000000,3112.000000,5.405750e+03,5.334000e+03
50%,5000.50000,3.949655e+05,4.251235e+05,2.719524e+06,40.000000,9.780000e+12,2004.000000,4.020000,2.115550e+04,2.383250e+04,1402.000000,391.000000,1163.000000,4894.000000,8.269500e+03,8.836000e+03
75%,7500.25000,9.382225e+06,9.636112e+06,1.451775e+07,67.000000,9.780000e+12,2011.000000,4.180000,4.105350e+04,4.591500e+04,2744.250000,885.000000,2353.250000,9287.000000,1.602350e+04,1.730450e+04
max,10000.00000,3.328864e+07,3.553423e+07,5.639960e+07,3455.000000,9.790000e+12,2017.000000,4.820000,4.780653e+06,4.942365e+06,155254.000000,456191.000000,436802.000000,793319.000000,1.481305e+06,3.011543e+06


In [2]:
books = books.loc[books['average_rating'] >= 3.5]

print(books.shape)
print(books.columns)

(9661, 23)
Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')


In [3]:
#ratings = pd.read_csv('ratings.csv', encoding = 'ISO-8859-1')
#ratings.head()

In [4]:
book_tags = pd.read_csv('book_tags.csv', encoding = 'ISO-8859-1')
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [5]:
tags = pd.read_csv('tags.csv')
# drop the 'to-read', 'favorites' and 'currently-reading' tag
tags.drop([30574, 11557, 8717], inplace = True)
print(tags.tail())

       tag_id    tag_name
34247   34247   Ｃhildrens
34248   34248   Ｆａｖｏｒｉｔｅｓ
34249   34249       Ｍａｎｇａ
34250   34250      ＳＥＲＩＥＳ
34251   34251  ｆａｖｏｕｒｉｔｅｓ


In [6]:
book_tags_df = pd.merge(book_tags, tags, on = 'tag_id', how = 'inner').sort_values(by = 'count', ascending = False)

#row = [i for i, row in book_tags_df.iterrows() if row['tag_name'] == 'to-read']
#book_tags_df.drop(row, inplace = True)
book_tags_df.head()
book_tags_df.shape

(970272, 4)

In [7]:
#to_read = pd.read_csv('to_read.csv')
#to_read.head()

In [8]:
# author based recommender

tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 3), min_df = 0, stop_words = 'english')
#tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(books['authors'])
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
#Construct a reverse map of indices and book titles
indices = pd.Series(books.index, index = books['title']).drop_duplicates()

In [10]:
# Function that takes in book titles as input and outputs most similar books
def get_recommendations(title, cosine_sim = cosine_sim):
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return books['title'].iloc[book_indices]

In [11]:
books['title'].head(20)

0               The Hunger Games (The Hunger Games, #1)
1     Harry Potter and the Sorcerer's Stone (Harry P...
2                               Twilight (Twilight, #1)
3                                 To Kill a Mockingbird
4                                      The Great Gatsby
5                                The Fault in Our Stars
6                                            The Hobbit
7                                The Catcher in the Rye
8                 Angels & Demons  (Robert Langdon, #1)
9                                   Pride and Prejudice
10                                      The Kite Runner
11                            Divergent (Divergent, #1)
12                                                 1984
13                                          Animal Farm
14                            The Diary of a Young Girl
15     The Girl with the Dragon Tattoo (Millennium, #1)
16                 Catching Fire (The Hunger Games, #2)
17    Harry Potter and the Prisoner of Azkaban (

In [12]:
# tag based recommender

books_with_tags = pd.merge(books, book_tags_df, left_on = 'book_id', right_on = 'goodreads_book_id', how = 'inner')
books_with_tags['tag_name'] = books_with_tags['tag_name'].astype('category')
#books_with_tags.info()
tfidf1 = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 3), min_df = 0, stop_words = 'english')
tfidf_matrix1 = tfidf1.fit_transform(books_with_tags['tag_name'].head(20000))
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [13]:
# tag and author based recommender

temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

,book_id,tag_name
0,1,fantasy young-adult fiction harry-potter books...
1,2,fantasy children children-s all-time-favorites...
2,3,fantasy young-adult fiction harry-potter books...
3,5,fantasy young-adult fiction harry-potter books...
4,6,fantasy young-adult fiction harry-potter owned...


In [14]:
books0 = pd.merge(books, temp_df, on = 'book_id', how = 'inner')
#books0.head()

In [15]:
# function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

In [16]:
books0['authors'].apply(clean_data)

0                                          suzannecollins
1                               j.k.rowling,marygrandprì©
2                                          stepheniemeyer
3                                               harperlee
4                                       f.scottfitzgerald
5                                               johngreen
6                                           j.r.r.tolkien
7                                            j.d.salinger
8                                                danbrown
9                                              janeausten
10                                         khaledhosseini
11                                           veronicaroth
12                   georgeorwell,erichfromm,celì¢lìïster
13                                           georgeorwell
14      annefrank,eleanorroosevelt,b.m.mooyaart-doubleday
15                                stieglarsson,regkeeland
16                                         suzannecollins
17            

In [17]:
def create_soup(x):
    return ''.join(x['authors']) + '' + ''.join(x['tag_name'])

In [18]:
books0['soup'] = books0.apply(create_soup, axis = 1)

In [19]:
# Use the CountVectorizer() instead of TF-IDF. 
# This is because one do not want to down-weight the presence of an author if he or she has written relatively more books.
# But will still fit_transform a tfidfvectorizer for comparison 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer(analyzer = 'word', ngram_range=(1, 3), min_df = 0, stop_words = 'english')
count_matrix = cv.fit_transform(books0['soup'])
cosine_sim0 = cosine_similarity(count_matrix, count_matrix)
#books0.reset_index()

In [20]:
tfidf2 = TfidfVectorizer(analyzer = 'word', ngram_range=(1, 3), min_df = 0, stop_words = 'english')
tfidf_matrix2 = tfidf2.fit_transform(books0['soup'])
cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [21]:
print('author-based recommendations:')
print(get_recommendations('The Great Gatsby'), '\n')
print('tag-based recommendations:')
print(get_recommendations('The Great Gatsby', cosine_sim1), '\n')
print('author and tag based recommendations using CountVectorizer:')
print(get_recommendations('The Great Gatsby', cosine_sim0), '\n')
print('author and tag based recommendations using tf-idf:')
print(get_recommendations('The Great Gatsby', cosine_sim2))

author-based recommendations:
1183                                  Tender Is the Night
2303                                This Side of Paradise
3254                             The Beautiful and Damned
3640                  The Curious Case of Benjamin Button
7408                                    The Short Stories
8683    The Billionaire's Obsession ~ Simon (The Billi...
1279                                           The Aeneid
922     The Alchemyst (The Secrets of the Immortal Nic...
1633    The Magician (The Secrets of the Immortal Nich...
2016    The Sorceress (The Secrets of the Immortal Nic...
Name: title, dtype: object 

tag-based recommendations:
97         The Girl Who Played with Fire (Millennium, #2)
197                                      The Color Purple
590       The Absolutely True Diary of a Part-Time Indian
813                          Tempted (House of Night, #6)
1095    America (The Book): A Citizen's Guide to Democ...
1204      Hard-Boiled Wonderland and the End

Author-based and tag-based recommenders have awful performance obviously. Although, when it comes to tag and author based recommender, the results of countvectorizer and tfidfvectorizer only have relatively small differences.

In [22]:
#print('author-based recommendations:')
#print(get_recommendations('Brave New World'), '\n')
#print('tag-based recommendations:')
#print(get_recommendations('Brave New World', cosine_sim1), '\n')
#print('author and tag based recommendations using CountVectorizer:')
#print(get_recommendations('Brave New World', cosine_sim0), '\n')
#print('author and tag based recommendations using TfidfVectorizer:')
#print(get_recommendations('Brave New World', cosine_sim2))

dict = {'CountVectorizer': [x for i,x in enumerate(get_recommendations('Animal Farm', cosine_sim0))], 
        'TfidfVectorizer': [x for i,x in enumerate(get_recommendations('Animal Farm', cosine_sim2))]}
results = pd.DataFrame(data = dict)
results

,CountVectorizer,TfidfVectorizer
0,Lord of the Flies,1984
1,The Great Gatsby,The Great Gatsby
2,1984,Animal Farm / 1984
3,Of Mice and Men,Keep the Aspidistra Flying
4,Brave New World,Lord of the Flies
5,The Catcher in the Rye,"Cry, the Beloved Country"
6,Fahrenheit 451,Of Mice and Men
7,The Old Man and the Sea,The Fall of the House of Usher
8,Silas Marner,A Modest Proposal
9,To Kill a Mockingbird,A Modest Proposal and Other Satirical Works
